In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
 
 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
# TODO:解释参数含义，在?处填入合适的参数
batch_size = 64
learning_rate = 0.001
num_epochs = 10
 
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])
 
# root可以换为你自己的路径
trainset = torchvision.datasets.CIFAR10(root='D:/Dataset/CIFAR-10', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
 
testset = torchvision.datasets.CIFAR10(root='D:/Dataset/CIFAR-10', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)

class InceptionA(nn.Module):
    def __init__(self,in_channels):
        super(InceptionA,self).__init__()
        self.branch1x1=nn.Conv2d(in_channels,8,kernel_size=1)
        self.branch1x1_bn=nn.BatchNorm2d(8)

        self.branch5x5_1=nn.Conv2d(in_channels,8,kernel_size=1)
        self.branch5x5_2=nn.Conv2d(8,12,kernel_size=5,padding=2)
        self.branch5x5_bn=nn.BatchNorm2d(12)

        self.branch3x3_1=nn.Conv2d(in_channels,8,kernel_size=1)
        self.branch3x3_2=nn.Conv2d(8,12,kernel_size=3,padding=1)
        self.branch3x3_3=nn.Conv2d(12,12,kernel_size=3,padding=1)
        self.branch3x3_bn=nn.BatchNorm2d(12)

        self.branch_pool=nn.Conv2d(in_channels,12,kernel_size=1)
        self.branch_pool_bn=nn.BatchNorm2d(12)

    def forward(self,x):
        branch1x1=self.branch1x1_bn(F.relu(self.branch1x1(x)))

        branch5x5=F.relu(self.branch5x5_1(x))
        branch5x5=self.branch5x5_bn(F.relu(self.branch5x5_2(branch5x5)))

        branch3x3=F.relu(self.branch3x3_1(x))
        branch3x3=F.relu(self.branch3x3_2(branch3x3))
        branch3x3=self.branch3x3_bn(F.relu(self.branch3x3_3(branch3x3)))

        branch_pool=F.avg_pool2d(x,kernel_size=3,stride=1,padding=1)
        branch_pool=self.branch_pool_bn(F.relu(self.branch_pool(branch_pool)))
        outputs=[branch1x1,branch5x5,branch3x3,branch_pool]
        return torch.cat(outputs,dim=1)


 
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        # TODO:这里补全你的网络层
        self.conv1=nn.Conv2d(3,32,kernel_size=3,padding=1)
        self.conv2=nn.Conv2d(32,64,kernel_size=3,padding=1)

        self.inception1=InceptionA(64)
        self.conv3=nn.Conv2d(44,88,kernel_size=1)
        self.inception2=InceptionA(88)
        self.pool=nn.MaxPool2d(2,2)
        
        self.fc1=nn.Linear(704,512)  
        self.fc2=nn.Linear(512,256)
        self.fc3=nn.Linear(256,10)

        self.dropout=nn.Dropout(0.5)
        
 
    def forward(self, x):
        # TODO:这里补全你的前向传播
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)  
        
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)  
        
        
        x = self.inception1(x)
        x = self.conv3(x)
        x = self.inception2(x)
        x = self.pool(x)
        
        x = x.view(x.size(0), -1)
        
        
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.dropout(x)
        x=self.fc3(x)
        
        return x
        
 
# TODO:补全
model = Network().to(device)
 
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)
 
def train():
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
 
            optimizer.zero_grad()
 
            outputs = model(inputs)
            loss = criterion(outputs, labels)
 
            loss.backward()
            optimizer.step()
 
            running_loss += loss.item()
 
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
 
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')
 
def test():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
 
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the 10000 test images: {accuracy:.2f}%')
 
if __name__ == "__main__":
    train()
    test()

Files already downloaded and verified
Files already downloaded and verified
Epoch [1/10], Loss: 1.3474, Accuracy: 51.52%
Epoch [2/10], Loss: 0.9635, Accuracy: 66.54%
Epoch [3/10], Loss: 0.7893, Accuracy: 72.50%
Epoch [4/10], Loss: 0.6795, Accuracy: 76.45%
Epoch [5/10], Loss: 0.5987, Accuracy: 79.18%
Epoch [6/10], Loss: 0.5214, Accuracy: 81.75%
Epoch [7/10], Loss: 0.4554, Accuracy: 84.16%
Epoch [8/10], Loss: 0.3937, Accuracy: 86.07%
Epoch [9/10], Loss: 0.3472, Accuracy: 87.93%
Epoch [10/10], Loss: 0.3071, Accuracy: 89.08%
Accuracy of the model on the 10000 test images: 74.94%
